<a href="https://colab.research.google.com/github/PedroPertusi/afd-monitor-jogadores/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !python -m pip install selenium
# !python -m pip install webdriver_manager
# !python -m pip install requests
# !python -m pip install BeautifulSoup4

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
def baixar_pagina(url):
  response = requests.get(url)
  response.encoding = "utf-8"
  html = response.text
  return html

In [3]:
def extrair_links_noticias_post(url):
  html = baixar_pagina(url)
  soup = BeautifulSoup(html, 'html.parser')
  title = soup.find_all('h2')
  links_noticias = []
  for t in title:
    if t.find('a').get('href'):
      links_noticias.append(t.find('a').get('href'))
  return links_noticias

In [4]:
def extrair_conteudo_noticia(url):
    html = baixar_pagina(url)
    soup = BeautifulSoup(html, 'html.parser')
    ps = soup.find_all('p', class_='content-text__container')

    textos = []
    for p in ps:
        # descarta parágrafos que contenham intertítulo ou lista
        if p.find('div', class_='content-intertitle') or p.find('ul'):
            continue
        # extrai texto limpo (inclui <a>, <strong>, etc)
        textos.append(p.get_text(" ", strip=True))
    return textos

In [5]:
times = ["atletico-mg", "bahia", "botafogo", "bragantino", "ceara", "corinthians", "cruzeiro", "flamengo", "fluminense", "fortaleza",
         "gremio", "internacional", "juventude", "mirassol", "palmeiras", "santos", "sao-paulo", "sport", "vasco", "vitoria"]

conteudos_times = {}

In [6]:
base_url = "https://ge.globo.com/futebol/times/"

for t in times:
  time_url = base_url + t
  noticias = extrair_links_noticias_post(time_url)

  conteudos = []
  for n in noticias:
    conteudos.append(extrair_conteudo_noticia(n))

  pattern = re.compile(r'^\+ O ge')
  c = [" ".join(parts) for parts in conteudos if not pattern.match(" ".join(parts))]

  c = [i for i in c if i not in (" ", "")]

  conteudos_times[t] = c

In [7]:
records = [
    (time, noticia)
    for time, noticias in conteudos_times.items()
    for noticia in noticias
]

df_time_noticia = pd.DataFrame(records, columns=['time', 'noticia'])

In [8]:
df_time_noticia

,time,noticia
0,atletico-mg,O meia-atacante Dudu esteve na Cidade do Galo ...
1,atletico-mg,O torcedor do Atlético-MG matou a saudade da A...
2,atletico-mg,Os argentinos do Estudiantes de La Plata solta...
3,atletico-mg,"De ""time sem vergonha"" a ""o Galo é o time da v..."
4,atletico-mg,Após a vitória do Atlético-MG contra o Flumine...
...,...,...
80,vitoria,"Depois de uma longa espera, a estreia de Renat..."
81,vitoria,"Com um jogador a menos, o Vitória mostrou forç..."
82,vitoria,"De virada, Vitória bateu o Vasco por 2 a 1 , n..."
83,vitoria,Renato Kayzer teve uma daquelas noites ilumina...


In [9]:
df_time_noticia.to_csv("times_noticias.csv")